In [2]:
%pylab inline
import itertools
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    accuracy_score,
    auc,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
import rpy2.robjects as robjects
from sklearn.model_selection import train_test_split, StratifiedKFold
import os


class_names = np.array(["No event", "Met event"])


def load_file(name):
    return np.genfromtxt(name, delimiter=",", skip_header=1)


def load_train_and_test_parts():
    X_train = load_file("data/microarray_train.csv")
    X_test = load_file("data/microarray_test.csv")
    y_train = load_file("data/labels_train.csv")
    y_test = load_file("data/labels_test.csv")
    return X_train, X_test, y_train, y_test


def plot_confusion_matrix(
    axis, cm, classes, normalize=False, title="Confusion matrix", cmap=cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    im = axis.imshow(cm, interpolation="nearest", cmap=cmap, vmin=0, vmax=1)
    axis.set(title=title, xlabel="Predicted label", ylabel="True label")
    tick_marks = np.arange(len(classes))
    axis.set_xticks(tick_marks)
    axis.set_xticklabels(classes)
    axis.set_yticks(tick_marks)
    axis.set_yticklabels(classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        axis.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    return im


def plot_roc_curve(ax, clf, dataset, labels, title):

    probs = clf.predict_proba(dataset)[:, 1]
    fpr, tpr, _ = roc_curve(labels, probs)
    roc_auc = auc(fpr, tpr)

    ax.plot(fpr, tpr, label="ROC curve (area = {:.2f})".format(roc_auc))
    ax.plot([0, 1], [0, 1], linestyle="--")
    ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=title + " ( Accuracy = {:.3f} )".format(clf.score(dataset, labels)),
    )
    ax.legend(loc="lower right")


def fit_clf(clf, train_set, train_labels, test_set, test_labels, title):
    clf = clf.fit(train_set, train_labels)
    plot_clf_roc(clf, train_set, train_labels, test_set, test_labels, title)
    # plot_clf_cm(clf, train_set, train_labels, test_set, test_labels)
    return clf


def fit_clf_with_cross_val(clf, train_set, train_labels, test_set, test_labels, title):
    fig, (ax1, ax2) = subplots(
        nrows=1, ncols=2, sharex=True, sharey=True, figsize=(16, 8)
    )
    fig.suptitle(title)
    fit_clf_cv(ax1, clf, train_set, train_labels, title="Train")
    plot_roc_curve(ax2, clf, test_set, test_labels, title="Test")
    plt.show()


def fit_clf_cv(ax, clf, X, y, title):
    cv = StratifiedKFold(n_splits=5)

    tprs = []
    aucs = []
    accs = [] 
    mean_fpr = np.linspace(0, 1, 100)

    i = 0
    for train, test in cv.split(X, y):
        model = clf.fit(X[train], y[train])
        probas_ = model.predict_proba(X[test])
        acc = model.score(X[test], y[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        accs.append(acc)
        ax.plot(
            fpr, tpr, lw=1, alpha=0.3, label="ROC fold %d (AUC = %0.2f)" % (i, roc_auc)
        )

        i += 1
    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(title + r" ( Accuracy = %0.2f $\pm$ %0.2f )" % (np.mean(accs), np.std(accs)))
    ax.legend(loc="lower right")


def fit_clf_no_plot(clf, train_set, train_labels, test_set, test_labels):
    clf = clf.fit(train_set, train_labels)
    _, _, train_roc_score = calculate_roc_score(clf, train_set, train_labels)
    _, _, test_roc_score = calculate_roc_score(clf, test_set, test_labels)
    return clf, train_roc_score, test_roc_score


def plot_clf_cm(clf, train_set, train_labels, test_set, test_labels):
    test_labels_pred = clf.predict(test_set)
    train_labels_pred = clf.predict(train_set)
    test_cm = confusion_matrix(test_labels, test_labels_pred)
    train_cm = confusion_matrix(train_labels, train_labels_pred)
    fig, (ax1, ax2) = subplots(nrows=1, ncols=2, sharex=True, sharey=True)
    im = plot_confusion_matrix(
        ax1,
        train_cm,
        classes=class_names,
        normalize=True,
        title="Train confusion matrix",
    )
    im = plot_confusion_matrix(
        ax2, test_cm, classes=class_names, normalize=True, title="Test confusion matrix"
    )
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
    show()


def plot_clf_roc(clf, train_set, train_labels, test_set, test_labels, title):
    fig, (ax1, ax2) = subplots(
        nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 6)
    )
    fig.suptitle(title)
    plot_roc_curve(ax1, clf, train_set, train_labels, title="Train")
    plot_roc_curve(ax2, clf, test_set, test_labels, title="Test")
    plt.show()


def fit_models(
    train_set, train_labels, test_set, test_labels, plot_logit_weigths=False
):
    clf_logit = fit_clf(
        LogisticRegression(solver="liblinear", penalty="l1", C=0.3),
        train_set,
        train_labels,
        test_set,
        test_labels,
        "Logistics regression",
    )
    if plot_logit_weigths:
        plot_logit_weights(clf_logit, "Logistics regression coefficients")
    # clf_svm = fit_clf(SVC(gamma='scale', C=7, probability=True), train_set, train_labels, test_set, test_labels, 'SVM')
    clf_forest = fit_clf(
        RandomForestClassifier(max_depth=4, n_estimators=2000, min_samples_leaf=10),
        train_set,
        train_labels,
        test_set,
        test_labels,
        "Random forest",
    )
    return (clf_logit, clf_forest)


def plot_logit_weights(clf_logit, title):
    plt.figure()
    plt.title(title)
    plt.plot(np.arange(clf_logit.coef_.shape[1]), clf_logit.coef_[0])
    plt.show()


def plot_logit_weights_ax(ax, clf_logit, title):
    ax.set_title(title)
    ax.plot(np.arange(clf_logit.coef_.shape[1]), clf_logit.coef_[0])


## MLCC
def read_mlcc_result(filename, train_size):
    robjects.r["load"]("./mlcc_results/{}".format(filename))
    s, m, b = robjects.r["res"]
    segmentation = np.asarray(s)
    numb_clust = np.max(s)
    mBIC = np.asarray(m)
    b.names = robjects.r("0:{}".format(numb_clust - 1))
    bases = dict(zip(b.names, map(list, list(b))))
    dimensionalities = np.empty(numb_clust, dtype=np.int32)
    for i in range(numb_clust):
        dimensionalities[i] = len(bases[str(i)]) // train_size
    return segmentation - 1, mBIC, dimensionalities


def apply_mlcc_dim_reduction(X, segmentation, dimensionalities):
    numb_clust = dimensionalities.shape[0]
    X_reduced = np.empty((X.shape[0], 0))
    for i in range(numb_clust):
        cluster = X[:, segmentation == i]
        n_components = dimensionalities[i]
        if cluster.shape[1] < n_components:  # TODO - maybe mlcc shouldn't allow it
            print(
                "WARNING! Dimensionality of a cluster was greater than the number of variables. Ignoring this cluster."
            )
        else:
            X_reduced = np.concatenate(
                (X_reduced, PCA(n_components=n_components).fit_transform(cluster)),
                axis=1,
            )
    return X_reduced


def quantile_normalize(vec):
    quants = np.quantile(vec, [0.25, 0.5, 0.75])
    return (vec - quants[1]) / (quants[2] - quants[0])


def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

Populating the interactive namespace from numpy and matplotlib


In [5]:
class RandomLogisticsRegressions:
    def __init__(
        self,
        n_estimators=11,
        penalty="l2",
        tol=1e-4,
        C=1.0,
        solver="liblinear",
        n_variables=1000
    ):
        self.n_estimators = n_estimators
        self.estimators_ = [
            LogisticRegression(penalty=penalty, tol=tol, C=C, solver=solver)
            for x in np.arange(n_estimators)
        ]
        self.indices = []
        self.n_variables = n_variables

    def fit(self, X, y):
        self.indices = np.array(
            [
                np.random.choice(
                    np.arange(X.shape[1]), size=self.n_variables, replace=False
                )
                for x in np.arange(self.n_estimators)
            ]
        )
        for i in np.arange(self.n_estimators):
            self.estimators_[i].fit(X[:, self.indices[i, :]], y)
        return self

    def predict(self, X):
        models_predictions = np.array(
            [
                model.predict(X[:, self.indices[i, :]])
                for i, model in enumerate(self.estimators_)
            ]
        )
        mean_predictions = np.mean(models_predictions, axis=0)
        return np.round(mean_predictions)

    def predict_proba(self, X):
        models_probs = np.array(
            [
                model.predict_proba(X[:, self.indices[i, :]])
                for i, model in enumerate(self.estimators_)
            ]
        )
        probabilities = np.mean(models_probs, axis=0)
        return probabilities

    def score(self, X, y):
        predictions = self.predict(X)
        return accuracy_score(y, predictions)